## Calculate Sentiment with Finance-Dict

compare sentiment with stock increase

In [1]:
#immport basic DataScience Modules
#!! for pip-modul-list to install @see requirements.txt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['id', 'headline', 'stock', 'headline_cleaned', 'open', 'close']]
data = data[:500]
dictionary = pd.read_csv('../data/finance_dict.csv')[['Word', 'Negative', "Positive"]]
dictionary["Word"] = dictionary['Word'].apply(lambda word: str(word).lower())

In [3]:
data.sample()
dictionary.sample(10)

,Word,Negative,Positive
8278,bounciest,0,0
31124,gearbox,0,0
62490,refract,0,0
35268,homered,0,0
1680,airtight,0,0
32965,grumbler,0,0
44043,macrocosm,0,0
19282,deliberation,0,0
54340,peasants,0,0
74969,suppress,0,0


In [4]:
def stockDevelopement(row):
    stock_differenz = row['open'] - row['close']
    quotient = row['close']/row['open']
    if quotient < 0.90:
        return -1
    elif quotient > 1.1:
        return 1
    else: return 0

In [5]:
data["stockPriceIncreased"] = data.apply(stockDevelopement, axis=1)

In [6]:
data.sample(5)

,id,headline,stock,headline_cleaned,open,close,stockPriceIncreased
464,5277,Stocks That Hit 52-Week Lows On Thursday,AAME,"['stock.n.01', 'hit.v.01', 'low.n.01', 'thursd...",1.790,2.00,1
379,4885,Int'l Air Transport Association Sees Airline I...,AAL,"['air.n.01', 'conveyance.n.03', 'association.n...",15.785,15.58,0
326,4832,Treasury Secretary Mnuchin Says Will Update Tr...,AAL,"['treasury.n.01', 'secretary.n.01', 'say.n.01'...",12.640,12.51,0
191,1928,Alcoa Reports Q4 Earnings Miss,AA,"['report.n.01', 'net_income.n.01', 'girl.n.01']",20.330,20.18,0
361,4867,Mnuchin Says We Have The Ability To Take Equit...,AAL,"['state.v.01', 'ability.n.01', 'take.v.01', 'e...",10.900,10.25,-1


In [7]:
def getSentiment(hl):
    senti_score = 0

    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    # print(word_list)
    for word in word_list:
        # print(word)
        try:
            _word = dictionary[dictionary["Word"] == word]
            # print(_word["Positive"].values[0])
            # print(_word["Negative"].values[0])
        
            senti_score += _word["Positive"].values[0]
            senti_score -= _word["Negative"].values[0]
        except:
            pass
    # # print(senti_score)
    return senti_score / 2009


In [8]:
data["senti"] = data["headline_cleaned"].apply(getSentiment)

In [9]:
data.sample(10)

,id,headline,stock,headline_cleaned,open,close,stockPriceIncreased,senti
99,1835,Shares of several basic material companies are...,AA,"['share.n.01', 'several.s.01', 'basic.a.01', '...",12.38,12.59,0,0.000000
209,1947,"Alcoa Reports Added Actions On Pension, Other ...",AA,"['report.n.01', 'action.n.01', 'pension.n.01',...",19.97,19.76,0,-1.005475
133,1869,"Cramer Advises His Viewers On Alcoa, Xerox, Te...",AA,"['rede.v.02', 'spectator.n.01', 'xerox.n.01', ...",7.23,7.26,0,0.000000
271,4766,Shares of several airline companies are tradin...,AAL,"['share.n.01', 'several.s.01', 'airline.n.01',...",17.69,17.02,0,0.000000
183,1920,76 Biggest Movers From Yesterday,AA,"['large.a.01', 'mover.n.01', 'yesterday.n.01']",18.06,17.69,0,0.000000
410,4916,"American Air To Suspend All US, China Flights",AAL,"['american.n.01', 'air.n.01', 'suspend.v.01', ...",27.50,26.84,0,-1.000000
239,1977,Alcoa shares are trading lower after Goldman S...,AA,"['share.n.01', 'trade.v.01', 'goldman.n.01', '...",20.66,21.30,0,-1.000000
123,1859,Alcoa Sees 2020 Aluminum Segment Shipments 2.9...,AA,"['see.n.01', 'aluminum.n.01', 'section.n.04', ...",7.63,7.55,0,0.000000
212,1950,"Benzinga's Top Upgrades, Downgrades For Novemb...",AA,"['top.n.01', 'ascent.n.01', 'downgrade.n.01', ...",21.53,21.22,0,-1.000000
226,1964,Alcoa shares are trading higher despite the co...,AA,"['share.n.01', 'trade.v.01', 'high.a.01', 'com...",19.25,19.16,0,0.000000


In [10]:
def checkAccuracy(row):
    if row['stockPriceIncreased'] == row["senti"]:
        return 1
    else: return 0

In [11]:
data["accuracy"] = data.apply(checkAccuracy, axis=1)

In [12]:
numberRowsCorrect = data[data["accuracy"] == 1].shape[0]
numberRows = data.shape[0]
accuracy = numberRowsCorrect / numberRows
print("Von " + str(numberRows) + " wurden " + str(numberRowsCorrect) +" richtig erkannt. Das ergbit eine Accuracy von: " + str(accuracy*100)+"%")



Von 500 wurden 275 richtig erkannt. Das ergbit eine Accuracy von: 55.00000000000001%
